In [1]:
# COMECO (Importar os dados)

########################################
#         Data Import from Sheets      #
########################################
import gspread
import pandas as pd
from datetime import datetime, timedelta
import calendar
import matplotlib.pyplot as plt
from oauth2client.service_account import ServiceAccountCredentials
import smtplib
from email.message import EmailMessage
from dotenv import load_dotenv
import os

load_dotenv()
email_password = os.getenv('EMAIL_PASSWORD')

# Set up credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/lohan/PythonProjects/3 - Finances/finances-411402-17fda5da6f27.json', scope)
client = gspread.authorize(creds)


# Open Spreadsheet based on sheet key
spreadsheet_key = '1K-EDM6ETO7yH-x-yFHSdqgxGxTufGSboxYvvpZF1nKo'
spreadsheet = client.open_by_key(spreadsheet_key)


# Open specific Sheet based on title
sheet_title = 'Expenses'  
sheet = spreadsheet.worksheet(sheet_title)

# Extract all of the data
data = sheet.get_all_records()

# Convert from JSON to pandas Data Frame
data = pd.DataFrame(data)

all_food = data[data['Expense Type'].isin(['Restaurant/Fast Food', 'Groceries'])]

#SILVER1
expenses_data = data

expenses_data['Timestamp'] = pd.to_datetime(expenses_data['Timestamp'])

# Filtering data for 'Restaurant/Fast Food' and 'Groceries' as general "food" categories
filtered_data = expenses_data[expenses_data['Expense Type'].isin(['Restaurant/Fast Food', 'Groceries'])]

# Current date
current_date = datetime.now()

this_month = current_date.strftime("%B")


if current_date.weekday() >= 4:  # Friday = 4, Saturday = 5, Sunday = 6
    week_start = (current_date - timedelta(days=current_date.weekday() - 4)).replace(hour=0, minute=0, second=0)
else:
    week_start = (current_date - timedelta(days=current_date.weekday() + 3)).replace(hour=0, minute=0, second=0)

week_end = (week_start + timedelta(days=6)).replace(hour=23, minute=59, second=59)

# Filter data for the current week
current_week_data = filtered_data[(filtered_data['Timestamp'] >= week_start) & (filtered_data['Timestamp'] <= week_end)]

# Sum of expenses for the current week
current_week_expenses1 = current_week_data['Expense Amount'].sum()

# Calculate weekly average expenses on food
weekly_average_expenses1 = filtered_data.set_index('Timestamp').resample('W-Thu')['Expense Amount'].sum().mean()

# Days until the end of this week
days_until_end_of_week = (week_end - current_date).days

# Outputting the report
week_start_str = week_start.strftime('%Y-%m-%d')
week_end_str = week_end.strftime('%Y-%m-%d')


############################################################################

# Resampling data to a weekly basis and summing expenses by type
weekly_resampled_data = filtered_data.groupby([pd.Grouper(key='Timestamp', freq='W-Thu')]).sum().reset_index()

# Pivot the data to have expense types as columns for plotting
pivot_data = weekly_resampled_data.pivot_table(values='Expense Amount', index='Timestamp', aggfunc='sum')

average_spending = pivot_data.mean().sum()


############################################################################

filtered_data = expenses_data[expenses_data['Expense Type'].isin(['Restaurant/Fast Food'])]


# Filter data for the current week
current_week_data = filtered_data[(filtered_data['Timestamp'] >= week_start) & (filtered_data['Timestamp'] <= week_end)]

# Sum of expenses for the current week
current_week_expenses2 = current_week_data['Expense Amount'].sum()

# Calculate weekly average expenses on food
weekly_average_expenses2 = filtered_data.set_index('Timestamp').resample('W-Thu')['Expense Amount'].sum().mean()


############################################################################

############################################################################

# Filtering data for 'Restaurant/Fast Food' and 'Groceries' as general "food" categories
filtered_data = expenses_data[expenses_data['Expense Type'].isin(['Groceries'])]

# Filter data for the current week
current_week_data = filtered_data[(filtered_data['Timestamp'] >= week_start) & (filtered_data['Timestamp'] <= week_end)]


# Sum of expenses for the current week
current_week_expenses3 = current_week_data['Expense Amount'].sum()

# Calculate weekly average expenses on food
weekly_average_expenses3 = filtered_data.set_index('Timestamp').resample('W-Thu')['Expense Amount'].sum().mean()


############################################################################


# Filtering data for 'Restaurant/Fast Food' and 'Groceries' as general "food" categories
filtered_data = expenses_data[expenses_data['Expense Type'].isin(['Restaurant/Fast Food', 'Groceries'])]

month_start = current_date.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
next_month_start = (month_start + timedelta(days=calendar.monthrange(month_start.year, month_start.month)[1])).replace(day=1, hour=0, minute=0, second=0, microsecond=0)
month_end = (next_month_start - timedelta(days=1)).replace(hour=23, minute=59, second=59)

# Filter data for the current month
current_month_data = filtered_data[(filtered_data['Timestamp'] >= month_start) & (filtered_data['Timestamp'] <= month_end)]

# Sum of expenses for the current month
current_month_expenses1 = current_month_data['Expense Amount'].sum()

# Calculate monthly average expenses on food
monthly_average_expenses1 = filtered_data.set_index('Timestamp').resample('ME')['Expense Amount'].sum().mean()

# Days until the end of the current month
days_until_end_of_month = (month_end - current_date).days

# Outputting the report
month_start_str = month_start.strftime('%Y-%m-%d')
month_end_str = month_end.strftime('%Y-%m-%d')


############################################################################

# Filtering data for 'Restaurant/Fast Food' and 'Groceries' as general "food" categories
filtered_data = expenses_data[expenses_data['Expense Type'].isin(['Restaurant/Fast Food'])]

# Filter data for the current month
current_month_data = filtered_data[(filtered_data['Timestamp'] >= month_start) & (filtered_data['Timestamp'] <= month_end)]

# Sum of expenses for the current month
current_month_expenses2 = current_month_data['Expense Amount'].sum()

# Calculate monthly average expenses on food
monthly_average_expenses2 = filtered_data.set_index('Timestamp').resample('ME')['Expense Amount'].sum().mean()


###########################################################################

# Filtering data for 'Restaurant/Fast Food' and 'Groceries' as general "food" categories
filtered_data = expenses_data[expenses_data['Expense Type'].isin(['Groceries'])]

# Filter data for the current month
current_month_data = filtered_data[(filtered_data['Timestamp'] >= month_start) & (filtered_data['Timestamp'] <= month_end)]

# Sum of expenses for the current month
current_month_expenses3 = current_month_data['Expense Amount'].sum()

# Calculate monthly average expenses on food
monthly_average_expenses3 = filtered_data.set_index('Timestamp').resample('ME')['Expense Amount'].sum().mean()





################## EMAIL ##############################

tabela_semana = {
    ' ': ['Restaurants/Fast Food', 'Groceries', 'All Food'],
    'Actual': [f'${current_week_expenses2:.2f}', f'${current_week_expenses3:.2f}', f'${current_week_expenses1:.2f}'],
    'Average': [f'${weekly_average_expenses2:.2f}', f'${weekly_average_expenses3:.2f}', f'${weekly_average_expenses1:.2f}'],
    'Days Until End of Week': [days_until_end_of_week, days_until_end_of_week, days_until_end_of_week]
}

tabela_mes = {
    ' ': ['Restaurants/Fast Food', 'Groceries', 'All Food'],
    'Actual': [f'${current_month_expenses2:.2f}', f'${current_month_expenses3:.2f}', f'${current_month_expenses1:.2f}'],
    'Average': [f'${monthly_average_expenses2:.2f}', f'${monthly_average_expenses3:.2f}', f'${monthly_average_expenses1:.2f}'],
    'Days Until End of Month': [days_until_end_of_month, days_until_end_of_month, days_until_end_of_month]
}

tabela_mes = pd.DataFrame(tabela_mes)
tabela_semana = pd.DataFrame(tabela_semana)



css_styles = """
<style>
    .myDataFrame {
        width: 50%;
        border-collapse: collapse;
    }
    .myDataFrame, .myDataFrame th, .myDataFrame td {
        border: 1px solid black;
    }
    .myDataFrame th, .myDataFrame td {
        padding: 10px;
        text-align: left;
    }
    .myDataFrame tr:nth-child(even) {
        background-color: #f2f2f2;
    }
    .myDataFrame th {
        background-color: #4CAF50;
        color: white;
    }
    .myDataFrame td {
        text-align: center;
    }
    .card {
        box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);
        transition: 0.3s;
        width: 20%;
        border-radius: 5px;
        display: inline-block;
        text-align: center;
        font-family: Arial, sans-serif;
        margin: 10px;
    }

    .container {
        padding: 2px 16px;
    }

    .number {
        font-size: 30px;
        color: black;
    }

    .green {
        background-color: #a4eda7; /* Green background */
    }

    .red {
        background-color: #f57e76; /* Red background */
    }
</style>
"""


tabela_semana_html = tabela_semana.to_html(index=False, classes='myDataFrame')
tabela_mes_html = tabela_mes.to_html(index=False, classes='myDataFrame')

available_for_the_day_week = weekly_average_expenses1 - current_week_expenses1
available_for_the_day_month = monthly_average_expenses1 - current_month_expenses1
available_for_the_per_day_week = (weekly_average_expenses1 - current_week_expenses1)/days_until_end_of_week
available_for_the_per_day_month = (monthly_average_expenses1 - current_month_expenses1)/days_until_end_of_month

card_class = "green" if available_for_the_day_week > 0 else "red"
card_class2 = "green" if available_for_the_day_month > 0 else "red"
card_class3 = "green" if available_for_the_per_day_week > 0 else "red"
card_class4 = "green" if available_for_the_per_day_month > 0 else "red"



msg = EmailMessage()
msg['Subject'] = f'Food Expense Report {datetime.today().date().strftime("%m-%d-%Y")}'
msg['From'] = 'lohan.rossi@hotmail.com'
msg['To'] = 'lohan.rossi@hotmail.com'
msg.set_content('This is an automated email. Please find the HTML version for better formatting.')

# Add the HTML version
msg.add_alternative(f"""\
<!DOCTYPE html>
<html>
    <head>
        {css_styles}
    </head>
    <body>
        <p>prototipo de um relatorio diario automatico, dados reais:<p>
        <br>
        <h1>FOOD EXPENSE REPORT</h1>

        <div class="card {card_class}">
            <div class="container">
                <p style = "font-weight: bold;">Available for the Week</p>
                <p class="number">${(available_for_the_day_week):.2f}</p>
                
            </div>
        </div>

        <div class="card">
            <div class="container">
                <p style = "font-weight: bold;">Days left for the Week</p>
                <p class="number">{days_until_end_of_week}</p>
                
            </div>
        </div>

        <div class="card {card_class3}">
            <div class="container">
                <p style = "font-weight: bold;">Available per Day</p>
                <p class="number">${(available_for_the_per_day_week):.2f}</p>
                
            </div>
        </div>

        <br>

        <div class="card {card_class2}">
            <div class="container">
                <p style = "font-weight: bold;">Available for the Month</p>
                <p class="number">${(available_for_the_day_month):.2f}</p>
            </div>
        </div>

        <div class="card">
            <div class="container">
                <p style = "font-weight: bold;">Days left for the Month</p>
                <p class="number">{days_until_end_of_month}</p>
            </div>
        </div>

        <div class="card {card_class4}">
            <div class="container">
                <p style = "font-weight: bold;">Available per Day</p>
                <p class="number">${(available_for_the_per_day_month):.2f}</p>
            </div>
        </div>

        <h2>THIS WEEK: {week_start_str} to {week_end_str}</h2>
        {tabela_semana_html}
        <h2>THIS MONTH: {this_month}</h2>
        {tabela_mes_html}
        <br>
        <br>
        <br>
        <br>

        <div class="card">
            <div class="container">
                <p style = "font-weight: bold;">All Food Expenses since February, 2023</p>
                <p class="number">${all_food['Expense Amount'].sum():,.2f}</p>
            </div>
        </div>

    </body>
</html>
""", subtype='html')

smtp_server = 'smtp.office365.com'
port = 587  # For TLS
username = 'lohan.rossi@hotmail.com'
password = email_password

# Connect to the server
server = smtplib.SMTP(smtp_server, port)
server.starttls()  # Upgrade the connection to secure
server.login(username, password)

# Send the email
server.send_message(msg)
server.quit()

C:\Users\lohan\AppData\Local\Temp\ipykernel_12180\3605836642.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


(221, b'2.0.0 Service closing transmission channel')